# Machine Learning for Predecttive Maintenance

## Introduction

### Predictive maintenance is a proactive approach to maintaining equipment by predicting potential failures before they occur. This helps in reducing unplanned downtime, minimizing maintenance costs, 
### and extending the lifespan of the equipment. In this notebook, we will demonstrate the implementation of a predictive maintenance model using both historical and real-time data.

### Due to the lack of existing real-world data, we will generate synthetic data to simulate the operating conditions and potential failures of industrial transformers and company laptops. The synthetic data will include various features such as temperature, vibration, load, ambient temperature, and humidity for transformers, and CPU usage, memory usage, disk health, and uptime for laptops.

### The main steps in this notebook include:
#### 1. **Data Generation:** Creating synthetic data that mimics real-world scenarios, including both normal operating conditions and potential failure conditions.
#### 2. **Data Preprocessing:** Cleaning and transforming the data to make it suitable for model training.
#### 3. **Feature Engineering:** Creating additional features that help improve the predictive power of the model.
#### 4. **Model Training:** Training machine learning models using the synthetic data to predict equipment failures.
#### 5. **Real-time Data Integration:** Simulating real-time data collection and feeding it into the model for continuous predictions.
#### 6. **Monitoring and Alerting:** Setting up mechanisms to monitor model predictions and generate alerts for potential failures.

### By the end of this notebook, we aim to build a robust predictive maintenance system that leverages both historical and real-time data to accurately predict equipment failures.
### This system will enable proactive maintenance, thereby improving reliability and efficiency in industrial and IT environments.

## Let's get started!


## Data Augmentation

### Transformer Data Augmentation

In [ ]:
import numpy  as np
import pandas as pd

# Define the number of samples
numSamples = 10000

# Generate normal operating conditions
normalTemperature = np.random.uniform(60, 80, numSamples)
normalVibration = np.random.uniform(0, 0.1, numSamples)
normalLoad = np.random.uniform(30, 70, numSamples)
normalAmbient_temp = np.random.uniform(20, 40, numSamples)
normalHumidity = np.random.uniform(30, 70, numSamples)

# Generate abnormal conditions (failures)
abnormalTemperature = np.random.uniform(80, 100, int(numSamples * 0.1))
abnormalVibration = np.random.uniform(0.1, 0.5, int(numSamples * 0.1))
abnormalLoad = np.random.uniform(70, 100, int(numSamples * 0.1))
abnormalAmbientTemp = np.random.uniform(40, 60, int(numSamples * 0.1))
abnormalHumidity = np.random.uniform(70, 100, int(numSamples * 0.1))

# Concatenate normal and abnormal conditions
temperature = np.concatenate((normalTemperature, abnormalTemperature))
vibration = np.concatenate((normalVibration, abnormalVibration))
load = np.concatenate((normalLoad, abnormalLoad))
ambientTemp = np.concatenate((normalAmbient_temp, abnormalAmbientTemp))
humidity = np.concatenate((normalHumidity, abnormalHumidity))

# Creating labels
labels = np.concatenate((np.zeros(numSamples), np.ones(int(numSamples * 0.1))))

# Create a DataFrame
data = pd.DataFrame({
    'temperature': temperature,
    'vibration': vibration,
    'load': load,
    'ambient_temp': ambientTemp,
    'humidity': humidity,
    'failure': labels
})

# Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

data.to_csv('synthetic_transformer_data.csv', index=False)

print(data.head())


### Laptop Data Augmentation

In [ ]:
import numpy as np
import pandas as pd

# Define the number of samples
numSamples = 10000

# Generate normal operating conditions
normalCpuUsage = np.random.uniform(10, 50, numSamples)
normalMemoryUsage = np.random.uniform(30, 70, numSamples)
normalDiskHealth = np.random.uniform(80, 100, numSamples)
normalBatteryHealth = np.random.uniform(70, 100, numSamples)
normalUptime = np.random.uniform(0, 100, numSamples)

# Generate abnormal conditions (crashes)
abnormalCpuUsage = np.random.uniform(50, 100, int(numSamples * 0.1))
abnormalMemoryUsage = np.random.uniform(70, 100, int(numSamples * 0.1))
abnormalDiskHealth = np.random.uniform(0, 80, int(numSamples * 0.1))
abnormalBatteryHealth = np.random.uniform(0, 70, int(numSamples * 0.1))
abnormalUptime = np.random.uniform(100, 200, int(numSamples * 0.1))

# Concatenate normal and abnormal conditions
cpuUsage = np.concatenate((normalCpuUsage, abnormalCpuUsage))
memoryUsage = np.concatenate((normalMemoryUsage, abnormalMemoryUsage))
diskHealth = np.concatenate((normalDiskHealth, abnormalDiskHealth))
batteryHealth = np.concatenate((normalBatteryHealth, abnormalBatteryHealth))
uptime = np.concatenate((normalUptime, abnormalUptime))

# Create labels
labels = np.concatenate((np.zeros(numSamples), np.ones(int(numSamples * 0.1))))

# Create a DataFrame
laptop_data = pd.DataFrame({
    'cpu_usage': cpuUsage,
    'memory_usage': memoryUsage,
    'disk_health': diskHealth,
    'battery_health': batteryHealth,
    'uptime': uptime,
    'system_crash': labels
})

# Shuffle the data
laptop_data = laptop_data.sample(frac=1).reset_index(drop=True)

laptop_data.to_csv('synthetic_laptop_data.csv', index=False)

print(laptop_data.head())